In [1]:
import io
import os

import re
import numpy as np
import pandas as pd
from collections import namedtuple

import shutil
from os import path, listdir
from nltk import word_tokenize
import itertools
import pickle
import glob

#from utils import Config
import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/home/mcaserta/research/matheur/utils.py'>

In [1]:
from gensim.models import doc2vec
#import logging
from gensim.models import Word2Vec, KeyedVectors
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [1]:
# download the file questions-words.txt to be used for comparing word embeddings
!wget https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt


converted 'https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt' (ANSI_X3.4-1968) -> 'https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt' (UTF-8)
--2019-03-18 08:23:50--  https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: 'questions-words.txt.1'

questions-words.txt 100%[=====================>] 589.80K  --.-KB/s   in 0.05s  

2019-03-18 08:23:50 (10.9 MB/s) - 'questions-words.txt.1' saved [603955/603955]



In [2]:
def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)


# Accuracy of different fasttext models

In [10]:
dims = [100, 200, 300]
lrs  = [0.05, 0.1, 0.15, 0.2]
epochs = [50, 100]
NwordNgrams = [1,2,3]

for dim in dims:
    for lr in lrs:
        for epoch in epochs:
            for ngrams in NwordNgrams:
                params = {}
                filename = "models/ft.model." + str(dim) + "." + str(lr) + "." + str(epoch) + "." + str(ngrams)
                print("-"*80)
                print("Building ... ", filename)
                #!fasttext skipgram -input myInput.txt -dim dim -lr lr -epoch epoch -wordNgrams ngrams -output filename
                !fasttext skipgram -input myInput.txt -dim {dim} -lr {lr} -epoch {epoch} -wordNgrams {ngrams} -output {filename}
                print("Done. Model written as {0}\n".format(filename))


--------------------------------------------------------------------------------
Building ...  models/ft.model.100.0.05.50.1
Read 7M words
Number of words:  20254
Number of labels: 0
Progress: 100.0% words/sec/thread:   53586 lr:  0.000000 loss:  0.781676 ETA:   0h 0m
Done. Model written as models/ft.model.100.0.05.50.1

--------------------------------------------------------------------------------
Building ...  models/ft.model.100.0.05.50.2
Read 7M words
Number of words:  20254
Number of labels: 0
Progress: 100.0% words/sec/thread:   53387 lr:  0.000000 loss:  0.761422 ETA:   0h 0m
Done. Model written as models/ft.model.100.0.05.50.2

--------------------------------------------------------------------------------
Building ...  models/ft.model.100.0.05.50.3
Read 7M words
Number of words:  20254
Number of labels: 0
Progress: 100.0% words/sec/thread:   52765 lr:  0.000000 loss:  0.760370 ETA:   0h 0m 0.021064 loss:  1.108707 ETA:   0h 4m  0h 3m
Done. Model written as models/ft.model.1

Read 7M words
Number of words:  20255
Number of labels: 0
Progress:   1.4% words/sec/thread:   48891 lr:  0.197106 loss: 15.613854 ETA:   0h21mterminate called after throwing an instance of 'std::runtime_error'
  what():  Encountered NaN.
terminate called recursively
terminate called recursively
terminate called recursively
terminate called recursively
Aborted
Done. Model written as models/ft.model.100.0.2.100.1

--------------------------------------------------------------------------------
Building ...  models/ft.model.100.0.2.100.2
Read 7M words
Number of words:  20255
Number of labels: 0
Progress:   1.4% words/sec/thread:   49876 lr:  0.197111 loss: 15.530802 ETA:   0h21mterminate called after throwing an instance of 'std::runtime_error'
  what():  Encountered NaN.
terminate called recursively
terminate called recursively
terminate called recursively
terminate called recursively
terminate called recursively
terminate called recursively
terminate called recursively
terminate called

Progress: 100.0% words/sec/thread:   25424 lr:  0.000000 loss:  0.586411 ETA:   0h 0m ETA:   0h38m37m 84.7% words/sec/thread:   25399 lr:  0.015310 loss:  0.689436 ETA:   0h 6m
Done. Model written as models/ft.model.300.0.1.100.2

--------------------------------------------------------------------------------
Building ...  models/ft.model.300.0.1.100.3
Read 7M words
Number of words:  20255
Number of labels: 0
Progress: 100.0% words/sec/thread:   25777 lr:  0.000000 loss:  0.595989 ETA:   0h 0mh34m
Done. Model written as models/ft.model.300.0.1.100.3

--------------------------------------------------------------------------------
Building ...  models/ft.model.300.0.15.50.1
Read 7M words
Number of words:  20255
Number of labels: 0
Progress:   3.8% words/sec/thread:   25521 lr:  0.144279 loss: 15.277019 ETA:   0h20mterminate called after throwing an instance of 'std::runtime_error'
  what():  Encountered NaN.
terminate called recursively
Aborted
Done. Model written as models/ft.model.30

Now, fine tuning, based on the best results obtained below.

In [14]:
!fasttext skipgram -input myInput.txt -dim 200 -lr 0.01 -epoch 50 -wordNgrams 3 -output models/ft.model.200.0.01.50.3

Read 7M words
Number of words:  20255
Number of labels: 0
Progress: 100.0% words/sec/thread:   32649 lr:  0.000000 loss:  0.738046 ETA:   0h 0m


In [5]:
accuracies = []
word_analogies_file = 'questions-words.txt'

model = KeyedVectors.load_word2vec_format("models/ft.model.200.0.01.50.3.vec")
#model = KeyedVectors.load_word2vec_format("models/ft.model.vec")
print('Accuracy for FastText (with n-grams):')
accuracies.append(print_accuracy(model, word_analogies_file))


Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 116/797, Accuracy: 14.55%
Syntactic: 1489/4479, Accuracy: 33.24%



In [9]:
!fasttext test 'models/ft.model.bin' /home/mcaserta/opt/fastText-0.2.0/test.txt

Test file cannot be opened!


In [11]:
listOfModels = glob.glob(path.join(utils.Config.prefix,utils.Config.models_dir,"ft.model*.vec*"))
print("Comparing ", len(listOfModels), " fasttext models.")
errors = []
for modelName in listOfModels:
    #listOfModels = []
    #modelName = "/home/mcaserta/research/nlp/matheur/models/ft.model.300.005.0.vec"
    word_analogies_file = 'questions-words.txt'
    accuracies = []

    try:
        print("\n","-"*80)
        print('\nLoading FastText embeddings - Model :: ', modelName)
        print("-"*80)
        model = KeyedVectors.load_word2vec_format(modelName)
        print('Accuracy for FastText (with n-grams):')
        accuracies.append(print_accuracy(model, word_analogies_file))
    except:
        print("Model ", model, "is faulty.")
        errors.append(model)
        

Comparing  43  fasttext models.

 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.model.200.0.1.100.1.vec
--------------------------------------------------------------------------------
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 276/797, Accuracy: 34.63%
Syntactic: 680/4479, Accuracy: 15.18%


 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.model.300.0.05.50.3.vec
--------------------------------------------------------------------------------
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 199/797, Accuracy: 24.97%
Syntactic: 838/4479, Accuracy: 18.71%


 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.mo

Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 206/797, Accuracy: 25.85%
Syntactic: 655/4479, Accuracy: 14.62%


 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.model.100.0.15.50.1.vec
--------------------------------------------------------------------------------
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 259/797, Accuracy: 32.50%
Syntactic: 654/4479, Accuracy: 14.60%


 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.model.200.0.1.50.3.vec
--------------------------------------------------------------------------------
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 223/797, Accuracy: 27.98%
Syntactic: 750/4479, Accuracy: 16.74%


 ----------------------------------------------------------------------

Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 202/797, Accuracy: 25.35%
Syntactic: 676/4479, Accuracy: 15.09%


 --------------------------------------------------------------------------------

Loading FastText embeddings - Model ::  /home/mcaserta/research/matheur/models/ft.model.300.0.1.50.2.vec
--------------------------------------------------------------------------------
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 244/797, Accuracy: 30.61%
Syntactic: 880/4479, Accuracy: 19.65%



# Accuracy of Word2Vec

In [5]:
accuracies = []
word_analogies_file = 'questions-words.txt'

fullname = path.join(utils.Config.models_dir,utils.Config.nameModel+".2")
print("Loading model ", fullname)
modelDoc2Vec = doc2vec.Doc2Vec.load(fullname)

print("The model contains {0} vectors.".format(len(modelDoc2Vec.docvecs)))
print('Accuracy for FastText (with n-grams):')
accuracies.append(print_accuracy(modelDoc2Vec.wv, word_analogies_file))

Loading model  models/doc2vec.model.2
The model contains 1806 vectors.
Accuracy for FastText (with n-grams):
Evaluating...


Semantic: 36/797, Accuracy: 4.52%
Syntactic: 317/4479, Accuracy: 7.08%

